<a href="https://colab.research.google.com/github/Da-ntas/Alura---ML---Valida-o/blob/main/Alura_ML_Valida%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"

dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25, stratify = y)

print("Treinameremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinameremos com 7500 elementos e testaremos com 2500 elementos


In [ ]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100
print("A acurácia do dummy stratified foi de %.2f%%" % acuracia)

A acurácia do dummy stratified foi de 58.00%


In [ ]:
from sklearn.tree import DecisionTreeClassifier

SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 71.92%


In [ ]:
def imprime_resultados(results):
  media = results['test_score'].mean()
  desvio_padrao = results['test_score'].std()
  print("Acurácia média: %.2f" % (media * 100))
  print("Acurácia intervalo: [%.2f, %.2f]" % ((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

# k-fold (validação cruzada)

In [ ]:
from sklearn.model_selection import cross_validate

SEED = 301
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv=5, return_train_score=False)

imprime_resultados(results)

Acurácia média: 75.78
Acurácia intervalo: [75.21, 76.35]


# k-fold com aleatorização

In [ ]:
from sklearn.model_selection import KFold

# sem shuffle

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits=10)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv=cv, return_train_score=False)

imprime_resultados(results)

Acurácia média: 75.78
Acurácia intervalo: [74.37, 77.19]


In [ ]:
from sklearn.model_selection import KFold

# com shuffle

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits=10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv=cv, return_train_score=False)

imprime_resultados(results)

Acurácia média: 75.76
Acurácia intervalo: [73.26, 78.26]


# Simular situação horrível de azar
Pode ser "azar" como pode ser uma proporção de exemplos desbalanceado entre as classes

In [ ]:
dados_azar = dados.sort_values("vendido", ascending=True)
x_azar = dados_azar[["preco", "idade_do_modelo", "km_por_ano"]]
y_azar = dados_azar["vendido"]

dados_azar.head()

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


In [ ]:
# sem shuffle

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits=10)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv=cv, return_train_score=False)

imprime_resultados(results)

Acurácia média: 57.84
Acurácia intervalo: [34.29, 81.39]


In [ ]:
# com shuffle

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits=10, shuffle=True)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv=cv, return_train_score=False)

imprime_resultados(results)

Acurácia média: 75.78
Acurácia intervalo: [72.30, 79.26]


In [ ]:
from sklearn.model_selection import StratifiedKFold

# com shuffle

SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits=10, shuffle=True)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv=cv, return_train_score=False)

imprime_resultados(results)

Acurácia média: 75.78
Acurácia intervalo: [73.55, 78.01]


# Gerando dados aleatórios

In [ ]:
np.random.seed(SEED)
dados["modelo"] = dados.idade_do_modelo + np.random.randint(-2, 3, size=(len(dados)))

dados.modelo = dados.modelo + abs(dados.modelo.min()) + 1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5


In [ ]:
dados.modelo.unique()

array([18, 24, 14,  6,  5, 13, 20, 19, 15,  2, 17, 12, 11, 16,  3,  7, 21,
       23, 10,  9, 22,  8,  4,  1])

# Testando validação cruzada com GroupKFold

In [ ]:
from sklearn.model_selection import GroupKFold

# com shuffle

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits=10)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv=cv, groups=dados.modelo, return_train_score=False)

imprime_resultados(results)

Acurácia média: 75.78
Acurácia intervalo: [73.67, 77.90]


# CrossVlidade com StandardScaler

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

scaler = StandardScaler()
scaler.fit(treino_x)
treino_x_escalado = scaler.transform(treino_x)
teste_x_escalado = scaler.transform(teste_x)

In [ ]:
from sklearn.svm import SVC

SEED = 301
np.random.seed(SEED)

modelo = SVC()
modelo.fit(treino_x_escalado, treino_y)
previsoes = modelo.predict(teste_x_escalado)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 74.40%


In [ ]:
cv = GroupKFold(n_splits=10)

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits=10)
modelo = SVC()
results = cross_validate(modelo, x_azar, y_azar, cv=cv, groups=dados.modelo, return_train_score=False)

imprime_resultados(results)

Acurácia média: 77.27
Acurácia intervalo: [74.35, 80.20]


# Pipeline

In [ ]:
from sklearn.pipeline import Pipeline

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([("transformacao", scaler), ("estimador", modelo)])

cv = GroupKFold(n_splits=10)
results = cross_validate(pipeline, x_azar, y_azar, cv=cv, groups=dados.modelo, return_train_score=False)
imprime_resultados(results)

Acurácia média: 76.68
Acurácia intervalo: [74.28, 79.08]
